In [11]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm import tqdm
import re
import time
import math

In [12]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.122 Safari/537.36'}

response = requests.get(url='https://wx.lianjia.com/zufang/',headers=headers)

url = 'https://wx.lianjia.com/zufang/{}/pg{}/'

In [14]:
area_dic = {
    '梁溪':'liangxi',
    '滨湖':'binhu',
    '锡山':'xishan',
    '惠山':'huishan',
    '新吴':'xinwu',
    '宜兴市':'yixingshi',
    '江阴市':'jiangyinshi'
}

In [15]:
# def re_match(re_pattern,string,errif=None):
#     try:
#         return re.findall(re_pattern,string)[0].strip()
#     except IndexError:
#         return errif

In [16]:
data = pd.DataFrame()

for key,value in area_dic.items():
    start_url = 'https://wx.lianjia.com/zufang/{}/'.format(value)
    html = requests.get(url=start_url,headers=headers).content.decode('utf-8')
    pattern = re.compile('<span class="content__title--hl">(.*?)</span>.*无锡',re.S)
    house_num = re.findall(pattern,html)[0].strip()
    print('{}:租房共{}套'.format(key,house_num))
    time.sleep(1)

    total_page = int(math.ceil(min(3000,int(house_num))/30.0))

    for i in tqdm(range(total_page),desc=key):
        html = requests.get(url=url.format(value,i+1),headers=headers).content.decode('utf-8')
        soup = BeautifulSoup(html,'lxml')

        info = soup.find_all(class_ = 'content__list--item--main')
        for item in info:
            info_dic = {}
            info_dic['area'] = key
            info_dic['title'] = re.findall(re.compile('.*<p class="content__list--item--title twoline">.*?href=.*?>(.*?)</a>',re.S),str(item))[0].strip()
            try:
                housesize = re.findall(re.compile('<p class="content__list--item--des">.*?<i>/</i>(.*?)㎡',re.S),str(item))[0].strip()
            except:
                housesize = re.findall(re.compile('<p class="content__list--item--des">(.*?)㎡',re.S),str(item))[0].strip()
            
            if '-' in housesize:
                low,high = housesize.split('-')
                info_dic['house_size'] = (int(low)+int(high))/2
            else:
                info_dic['house_size'] = int(housesize)

            try:
                housetypelist = re.findall(re.compile('<p class="content__list--item--des">.*?<i>/</i>.*?(\d)室(\d)厅(\d)卫.*?</p>',re.S),str(item))[0]   
            except:
                housetypelist = [0,0,0]
            info_dic['bedroon_num'] = int(housetypelist[0])
            info_dic['hall_num'] = int(housetypelist[1])
            info_dic['bathroom_num'] = int(housetypelist[2])
                
            price = re.findall(re.compile('<span class="content__list--item-price"><em>(.*?)</em>.*?</span>',re.S),str(item))[0]
            if '-' in price:
                low,high = price.split('-')
                info_dic['price'] = (int(low)+int(high))/2
            else:
                info_dic['price'] = float(price)
            #独栋·冠寓
            typeplace = (info_dic['title'].split(' ')[0]).split('·')
            info_dic['type'] = typeplace[0]
            info_dic['place'] = typeplace[1]

            if data.empty:
                data = pd.DataFrame(info_dic,index=[0])
            else:
                data = data.append(info_dic,ignore_index=True)

梁溪:租房共5496套
梁溪: 100%|██████████| 100/100 [02:00<00:00,  1.20s/it]
滨湖:租房共5139套
滨湖: 100%|██████████| 100/100 [02:01<00:00,  1.21s/it]
锡山:租房共1747套
锡山: 100%|██████████| 59/59 [01:15<00:00,  1.28s/it]
惠山:租房共1747套
惠山: 100%|██████████| 59/59 [01:17<00:00,  1.31s/it]
新吴:租房共2908套
新吴: 100%|██████████| 97/97 [02:27<00:00,  1.52s/it]
宜兴市:租房共4套
宜兴市: 100%|██████████| 1/1 [00:00<00:00,  1.27it/s]
江阴市:租房共728套
江阴市: 100%|██████████| 25/25 [00:37<00:00,  1.49s/it]


In [17]:
data.head()

,area,title,house_size,bedroon_num,hall_num,bathroom_num,price,type,place
0,梁溪,整租·惠东里 2室1厅 南,63.0,2,1,1,1400.0,整租,惠东里
1,梁溪,独栋·柚米国际社区 柚米(无锡三阳广场店) 北欧风 1室1厅,20.0,1,1,1,2072.5,独栋,柚米国际社区
2,梁溪,整租·中桥二村 2室1厅 南,57.0,2,1,1,1400.0,整租,中桥二村
3,梁溪,独栋·柚米国际社区 柚米(无锡三阳广场店) 一居清新风 1室1厅,19.5,1,1,1,2072.5,独栋,柚米国际社区
4,梁溪,整租·锡澄二村 3室1厅 南,77.0,3,1,1,1500.0,整租,锡澄二村


In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13157 entries, 0 to 13156
Data columns (total 9 columns):
area            13157 non-null object
title           13157 non-null object
house_size      13157 non-null float64
bedroon_num     13157 non-null int64
hall_num        13157 non-null int64
bathroom_num    13157 non-null int64
price           13157 non-null float64
type            13157 non-null object
place           13157 non-null object
dtypes: float64(2), int64(3), object(4)
memory usage: 925.2+ KB


In [19]:
#修改列名
data.rename(columns={'bedroon_num':'bedroom_num'},inplace=True) 

In [20]:
#查看室为零的数据,发现室卫厅都是0,全删掉
data[data['bedroom_num']==0]
data.drop(index=(data[data['bedroom_num']==0].index),inplace=True)

In [21]:
# 删除面积大于500 小于10的数据
data.drop(index=(data[(data['house_size']>500) | (data['house_size']<10)].index),inplace=True)
# 删除合租尺寸大于100的数据
data.drop(index=(data[(data['type']=='合租') & (data['house_size']>100)].index),inplace=True)
data.reset_index(drop=True,inplace=True)
data

,area,title,house_size,bedroom_num,hall_num,bathroom_num,price,type,place
0,梁溪,整租·惠东里 2室1厅 南,63.0,2,1,1,1400.0,整租,惠东里
1,梁溪,独栋·柚米国际社区 柚米(无锡三阳广场店) 北欧风 1室1厅,20.0,1,1,1,2072.5,独栋,柚米国际社区
2,梁溪,整租·中桥二村 2室1厅 南,57.0,2,1,1,1400.0,整租,中桥二村
3,梁溪,独栋·柚米国际社区 柚米(无锡三阳广场店) 一居清新风 1室1厅,19.5,1,1,1,2072.5,独栋,柚米国际社区
4,梁溪,整租·锡澄二村 3室1厅 南,77.0,3,1,1,1500.0,整租,锡澄二村
...,...,...,...,...,...,...,...,...,...
13104,江阴市,整租·中天花园 3室2厅 南,143.0,3,2,2,2600.0,整租,中天花园
13105,江阴市,整租·恒大御景 1室1厅 南,88.0,1,1,1,3000.0,整租,恒大御景
13106,江阴市,整租·金宸国际花园 2室2厅 南,100.0,2,2,1,2600.0,整租,金宸国际花园
13107,江阴市,整租·恒大中央广场 2室1厅 南,52.0,2,1,1,2000.0,整租,恒大中央广场


In [22]:
import pymysql
#连接数据库 127.0.0.1
conn = pymysql.connect(host='localhost',user='root',passwd='123456',db='Renting',port=3306,charset='utf8mb4')
#游标对象(操纵数据库必须使用游标提供了很多操作数据库的方法),比如执行sql
cursor = conn.cursor()
for i in range(0,data['area'].count()):
    #执行sql语句
    sql = "insert into Renting values('{}','{}','{}','{}','{}','{}','{}','{}','{}')".format(data['area'][i],data['title'][i],data['house_size'][i],data['bedroom_num'][i],data['hall_num'][i],data['bathroom_num'][i],data['price'][i],data['type'][i],data['place'][i])
    #执行sql语句
    cursor.execute(sql)
    #提交事务(查询可以不用使用)
    conn.commit()
#先关闭游标
cursor.close()
#关闭数据库
conn.close()

In [23]:
# 保存到excel
data.to_csv('Renting.xlsx',index=False)

In [24]:
from pyecharts import options as opts
from pyecharts.charts import *
from pyecharts.commons.utils import JsCode
from jieba import posseg as psg
import collections

In [25]:
#1室尺寸价格散点图
# 删除整租室大于1的数据
data1 = data.drop(index=(data[(data['type']=='整租')&(data['bedroom_num']>1)].index))
data1.reset_index(drop=True,inplace=True)
data1.describe()

,house_size,bedroom_num,hall_num,bathroom_num,price
count,3652.000000,3652.000000,3652.000000,3652.000000,3652.000000
mean,44.256024,1.015608,0.819825,0.957284,1633.907174
std,17.926603,0.185853,0.435184,0.241728,603.955088
min,10.000000,1.000000,0.000000,0.000000,300.000000
25%,33.000000,1.000000,1.000000,1.000000,1200.000000
50%,45.000000,1.000000,1.000000,1.000000,1650.000000
75%,52.000000,1.000000,1.000000,1.000000,2000.000000
max,400.000000,4.000000,3.000000,2.000000,10000.000000


In [26]:
scatter = (Scatter(init_opts=opts.InitOpts(theme='dark'))
           .add_xaxis(data1['house_size'])
           .add_yaxis('房租',data1['price'])
           .set_series_opts(label_opts=opts.LabelOpts(is_show=False),markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_='max',name='最大值'),]))
           .set_global_opts(legend_opts=opts.LegendOpts(is_show=False),#不显示图例
                            title_opts=opts.TitleOpts(title='无锡租房 房租-尺寸 散点图'),
                            xaxis_opts=opts.AxisOpts(
                                name='尺寸',
                                # 设置坐标轴为数值类型
                                type_='value',
                                # 不显示分割线
                                splitline_opts=opts.SplitLineOpts(is_show=False)),
                            yaxis_opts=opts.AxisOpts(
                                name='总价',
                                name_location='middle',
                                type_='value',
                                # 默认为False表示起始为0
                                is_scale=True,
                                splitline_opts=opts.SplitLineOpts(is_show=False),),
                            # 通过颜色来度量房租
                            visualmap_opts=opts.VisualMapOpts(is_show=True,type_='color',min_=0,max_=3000)))
scatter.render_notebook()

In [27]:
data1[data1['price']==10000]

,area,title,house_size,bedroom_num,hall_num,bathroom_num,price,type,place
3440,新吴,整租·金源国际大厦 1室0厅 南,220.0,1,0,0,10000.0,整租,金源国际大厦


In [28]:
# 各行政区1室租房均价
data2 = data.copy()
# 整租大于一室的房租除以室的个数
data2.loc[(data2['type']=='整租')&(data2['bedroom_num']>1),['price']] = (data2[(data2['type']=='整租')&(data2['bedroom_num']>1)]['price']) / (data2[(data2['type']=='整租')&(data2['bedroom_num']>1)]['bedroom_num'])
data2 = data2.groupby(['area'])['price'].mean().reset_index()
data2['area'] = data2['area'].map(lambda x:x+'区' if (x!='江阴市')&(x!='宜兴市') else x)
data2

,area,price
0,宜兴市,898.333333
1,惠山区,988.502574
2,新吴区,1365.170284
3,梁溪区,1191.139319
4,江阴市,989.993084
5,滨湖区,1184.373066
6,锡山区,955.655695


In [29]:
data_pair = [(row['area'],round(row['price'],1)) for _,row in data2.iterrows()]

map_ = (Map(init_opts=opts.InitOpts(theme='dark'))
       .add('房租均价',data_pair,'无锡',is_roam=False)
       .set_series_opts(label_opts=opts.LabelOpts(is_show=Tree))
       .set_global_opts(
           title_opts=opts.TitleOpts(title='无锡各地区租房均价'),
           visualmap_opts=opts.VisualMapOpts(min_=800,max_=1500)
       ))
map_.render_notebook()

In [30]:
# 均价最高10小区
data3 = data.copy()
# 整租大于一室的房租除以室的个数
data3.loc[(data3['type']=='整租')&(data3['bedroom_num']>1),['price']] = (data3[(data3['type']=='整租')&(data3['bedroom_num']>1)]['price']) / (data3[(data3['type']=='整租')&(data3['bedroom_num']>1)]['bedroom_num'])
data3 = data3.groupby(['place'])['price'].agg(['mean','count']).reset_index()

# key=lambda 元素: 元素[字段索引] 按照索引进行排序   
# reverse=True从大到小排序 
data_pair = sorted([(row['place'],round(row['mean'],1)) if row['count']>=3 else (0,0) for _,row in data3.iterrows()],key=lambda x:x[1],reverse=True)[:10]
data_pair

[('红豆柏雅居', 2585.7),
 ('绅华府', 2553.3),
 ('恒大财富中心', 2540.6),
 ('积水住宅裕沁湖畔庭', 2525.0),
 ('融创新天地', 2521.4),
 ('长江国际雅园', 2451.4),
 ('五爱人家', 2366.7),
 ('凯燕环球中心', 2366.7),
 ('茂业时代广场', 2342.0),
 ('万科东郡', 2241.3)]

In [31]:
bar = (Bar(init_opts=opts.InitOpts(theme='dark'))
       .add_xaxis([x[0] for x in data_pair[::-1]])
       .add_yaxis('1室租房均价',[x[1] for x in data_pair[::-1]])
       .set_series_opts(label_opts=opts.LabelOpts(is_show=True,
                                                position='insideRight',
                                                font_style='italic'),
                        itemstyle_opts=opts.ItemStyleOpts(
                                color=JsCode("""new echarts.graphic.LinearGradient(1, 0, 0, 0, 
                                             [{
                                                 offset: 0,
                                                 color: 'rgb(0,206,209)'
                                             }, {
                                                 offset: 1,
                                                 color: 'rgb(218,165,32)'
                                             }])""")
                        ))
        .set_global_opts(
            title_opts=opts.TitleOpts(title='无锡1室租房均价前10小区'),
            legend_opts=opts.LegendOpts(is_show=False),
            tooltip_opts=opts.TooltipOpts(formatter='{b}:{c}元'),
            xaxis_opts=opts.AxisOpts(min_=2000)
        )
        .reversal_axis())
bar.render_notebook()

In [32]:
# 户型分布
data4 = data.groupby(['bedroom_num','hall_num','bathroom_num'])['area'].count().reset_index()
data4['house_type'] = data4['bedroom_num'].astype('str') + '室' + data4['hall_num'].astype('str') + '厅' + data4['bathroom_num'].astype('str') + '卫'

data_pair = sorted([(row['house_type'],row['area']) for _,row in data4.iterrows()],key=lambda x:x[1],reverse=True)[:10]
data_pair

[('2室1厅1卫', 3074),
 ('1室1厅1卫', 2795),
 ('2室2厅1卫', 1695),
 ('3室2厅1卫', 1512),
 ('3室2厅2卫', 1184),
 ('3室1厅1卫', 1080),
 ('1室0厅1卫', 562),
 ('4室2厅2卫', 312),
 ('1室0厅0卫', 169),
 ('3室1厅2卫', 143)]

In [33]:
pie = (Pie(init_opts=opts.InitOpts(theme='dark'))
        .add('',data_pair,
            radius=['30%','75%'],
            rosetype='radius')
        .set_global_opts(title_opts=opts.TitleOpts(title='无锡租房 户型分布'),
                        legend_opts=opts.LegendOpts(is_show=False),
                        )
        .set_series_opts(label_opts=opts.LabelOpts(formatter='{b}:{d}%'))
        )
pie.render_notebook()

In [36]:
# 词云图
word_list = []
stop_words = ['南整']
string = str(''.join([i for i in data['title'] if isinstance(i,str)]))

words = psg.cut(string)
for x in words:
    if len(x.word) == 1:
        # print(x.word)
        pass
    elif x.flag in ('m','x'):
        # print(x.flag)
        pass
    elif x.word in stop_words:
        pass
    else:
        word_list.append(x.word)
        # print(word_list)

In [37]:
data_pair = collections.Counter(word_list).most_common(100)

wc = (WordCloud()
      .add('', data_pair, word_size_range=[20, 100], shape='triangle')
      .set_global_opts(title_opts=opts.TitleOpts(title="租房描述词云图"))
    )
wc.render_notebook()